In [1]:
%load_ext autoreload
%autoreload 2

# Typewriter: Single Tool

Let's see how to evaluate an agent's ability to use tools.

    A task where the agent must type a given string one letter at a time.

    In this variation of the task, the agent is given a single function,
    that takes a letter as an argument.

In [2]:
from langchain_benchmarks import clone_public_dataset, registry

In [3]:
task = registry["Tool Usage - Typewriter (1 tool)"]
task

Name,Tool Usage - Typewriter (1 tool)
Type,ToolUsageTask
Dataset ID,https://smith.langchain.com/public/59577193-8938-4ccf-92a7-e8a96bcf4f86/d
Description,"Environment with a single tool that accepts a single letter as input, and prints it on a piece of virtual paper. The objective of this task is to evaluate the ability of the model to use the provided tools to repeat a given input string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."


Clone the dataset associaetd with this task

In [4]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Typewriter (1 tool) already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/25850d74-d4e0-41ac-81a1-dfc78a79660b.


Let's build an agent that we can use for evaluation.

In [5]:
from langchain_benchmarks.tool_usage import agents

agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

# Let's test that our agent works
agent = agent_factory.create()
agent.invoke({"question": "abc"})

{'question': 'abc',
 'output': 'a, b, c',
 'intermediate_steps': [(AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'a'}, log="\nInvoking: `type_letter` with `{'letter': 'a'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "a"\n}', 'name': 'type_letter'}})]),
   'OK'),
  (AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'b'}, log="\nInvoking: `type_letter` with `{'letter': 'b'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "b"\n}', 'name': 'type_letter'}})]),
   'OK'),
  (AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'c'}, log="\nInvoking: `type_letter` with `{'letter': 'c'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "c"\n}', 'name': 'type_letter'}})]),
   'OK')],
 'state': 'abc'}

## Eval

Let's evaluate an agent now

In [6]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR

client = Client()

test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["gpt-3.5-turbo-16k"],
)

View the evaluation results for project 'test-crushing-trousers-4' at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/projects/p/df829a5b-a7f8-44bb-b68e-a21f9ea8330a?eval=true

View all tests for Dataset Tool Usage - Typewriter (1 tool) at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/25850d74-d4e0-41ac-81a1-dfc78a79660b
[------------------------------------------------->] 20/20
 Eval quantiles:
                                     0.25        0.5       0.75       mean  \
Intermediate steps correctness   0.000000   0.000000   0.000000   0.050000   
# steps / # expected steps       1.781250   2.500000   3.750000   2.825160   
Correct Final State              0.000000   0.000000   0.000000   0.050000   
correctness                      0.000000   0.000000   0.000000   0.050000   
execution_time                  35.480222  35.480222  35.480222  35.480222   

                                     mode  
Intermediate steps correctness   0.00

# Inspect

You can take a look at the underlying results.

In [7]:
import pandas as pd

df = test_run.to_dataframe()
df = pd.json_normalize(df.to_dict(orient="records"))

In [8]:
df["correctness"].mean()

0.05

In [16]:
df["num_expected_steps"] = df["reference.expected_steps"].apply(len)
df["actual_number_of_steps"] = df["output.intermediate_steps"].apply(len)

In [9]:
df.head()

,Intermediate steps correctness,# steps / # expected steps,Correct Final State,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,output.state,reference.state,reference.reference,reference.expected_steps
0,0,3.00,0,0,35.480222,a,a,b\nc,[(tool='type_letter' tool_input={'letter': 'a'...,abc,a,a,[type_letter]
1,0,7.50,0,0,35.480222,aa,aa,Agent stopped due to iteration limit or time l...,[(tool='type_letter' tool_input={'letter': 'a'...,aabacaaaaaaaaaa,aa,aa,"[type_letter, type_letter]"
2,0,5.00,0,0,35.480222,aaa,aaa,Agent stopped due to iteration limit or time l...,[(tool='type_letter' tool_input={'letter': 'a'...,aaaaaaaaaaaaaaa,aaa,aaa,"[type_letter, type_letter, type_letter]"
3,0,3.75,0,0,35.480222,aaaa,aaaa,Agent stopped due to iteration limit or time l...,[(tool='type_letter' tool_input={'letter': 'a'...,aaaaaaaaaaaaaaa,aaaa,aaaa,"[type_letter, type_letter, type_letter, type_l..."
4,1,1.00,1,0,35.480222,dog,dog,d\no,[(tool='type_letter' tool_input={'letter': 'd'...,dog,dog,dog,"[type_letter, type_letter, type_letter]"
